In [45]:
# Importando bibliotecas
import pandas as pd
from datetime import datetime
import time
import MetaTrader5 as mt5

tipo_ordem = 'V'

In [46]:
# Conectando-se ao MetaTrader 5
if not mt5.initialize():
    print("initialize() falied")
    mt5.shutdown()

In [47]:
# Obtendo Cotações
def get_ohlc(ativo, timeframe, n=10):
    ativo = mt5.copy_rates_from_pos(ativo, timeframe, 0, n)
    ativo = pd.DataFrame(ativo)
    ativo['time'] = pd.to_datetime(ativo['time'], unit = 's')
    ativo.set_index('time', inplace=True)
    return ativo

In [48]:
get_ohlc('EURUSD', mt5.TIMEFRAME_M5)

,open,high,low,close,tick_volume,spread,real_volume
time,,,,,,,
2021-12-23 10:25:00,1.13242,1.13269,1.13235,1.13256,282,1,0
2021-12-23 10:30:00,1.13256,1.13257,1.13212,1.13222,239,0,0
2021-12-23 10:35:00,1.13223,1.13226,1.13187,1.13205,290,0,0
2021-12-23 10:40:00,1.13205,1.13217,1.13182,1.13196,216,0,0
2021-12-23 10:45:00,1.13196,1.13196,1.13144,1.13165,200,1,0
2021-12-23 10:50:00,1.13166,1.13174,1.13125,1.13145,243,0,0
2021-12-23 10:55:00,1.13145,1.13147,1.13117,1.13147,218,1,0
2021-12-23 11:00:00,1.13147,1.13149,1.13106,1.13132,236,1,0
2021-12-23 11:05:00,1.13132,1.13152,1.13128,1.13134,246,0,0


In [49]:
mt5.symbol_info_tick('EURUSD')

Tick(time=1640257911, bid=1.13177, ask=1.13184, last=0.0, volume=0, time_msc=1640257911049, flags=6, volume_real=0.0)

In [50]:
# tempo = time.time() + 5

# while time.time() < tempo:
#     tick = mt5.symbol_info_tick('EURUSD')._asdict()
#     print(f"EURUSD:{tick['last']}, bid:{tick['bid']}, ask:{tick['ask']}")
#     time.sleep(0.5)

In [51]:
# Enviando Ordens

# Testando se o simbolo e valido
symbol = 'EURUSD'
symbol_info = mt5.symbol_info(symbol)
if symbol_info is None:
    print(symbol, "não encontrado")
    mt5.shutdown()
    quit()

In [52]:
# Adicionando o simbolo se nao estiver visivel no marktwatch
if not symbol_info.visible:
    print(symbol, "não esta visivel, tentando adicionar")
    if not mt5.symbol_select(symbol,True):
        print("symbol_select({}) failed, exit",symbol)
        mt5.shotdown()
        quit()

In [53]:
# Preparan a ordem Compra
if tipo_ordem == 'C':
    lot = 0.10
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).ask
    deviation = 20
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY,
        "price": price,
        "sl": price - 0.00050,
        "tp": price + 0.00050,
        "deviation": deviation,
        "magic": 234000,
        "comment": "python script open",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }

In [54]:
# Preparan a ordem Venda
if tipo_ordem == 'V':
    lot = 0.10
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).bid
    deviation = 20
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": price + 0.00050,
        "tp": price - 0.00050,
        "deviation": deviation,
        "magic": 234000,
        "comment": "python script open",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_RETURN,
    }

In [55]:
# Enviando a ordem
result = mt5.order_send(request)
# verificamos o resultado da execução
print(f"1. ordem enviada: {lot} de {symbol} ao preço de {price} com desvio de {deviation}")
if result.retcode != mt5.TRADE_RETCODE_DONE:
    print("2. envio de ordem falhou, retcode={}".format(result.retcode))
    # solicitamos o resultado na forma de dicionario e exibimos elementos
    result_dict = result._asdict()
    for field in result_dict.keys():
        print(f"  {field}={result_dict[field]}")
        # Se esta for uma estrutura de uma solicitação de negociação
        if field=="request":
            traderequest_dict = result_dict[field]._asdict()
            for tradereq_filed in traderequest_dict:
                print(f"  traderequest: {tradereq_filed}")

    print("shutdown() and quit")
    mt5.shutdown()
    quit()

1. ordem enviada: 0.1 de EURUSD ao preço de 1.13177 com desvio de 20
